In [8]:
import panel as pn
import hvplot.pandas  # noqa
import pandas as pd
import geopandas as gpd
from pathlib import Path

pn.extension("tabulator", throttled=True)


def load_data():
    """
    Load the processed geospatial dataset that already includes:
    - geometry
    - accessibility surfaces (E2SFCA)
    - Medicaid-only accessibility
    - equity gap
    - priority index
    - social vulnerability index (SVI)
    - travel time to nearest provider
    - cluster labels
    - machine learning predictions
    """
    data_path = Path("data") / "processed" / "final_accessibility.geojson"
    gdf = gpd.read_file(data_path)

    # Expected columns, adjust if your names differ
    expected_numeric = [
        "e2sfca_all",
        "e2sfca_medicaid",
        "equity_gap",
        "priority_index",
        "svi",
        "travel_time_nn",
        "ml_prediction",
    ]

    for col in expected_numeric:
        if col in gdf.columns:
            gdf[col] = pd.to_numeric(gdf[col], errors="coerce")

    # Basic safety for cluster label
    if "cluster_label" in gdf.columns:
        gdf["cluster_label"] = gdf["cluster_label"].astype(str)
    else:
        # Fallback if you used another name
        raise ValueError(
            "Column 'cluster_label' not found. "
            "Please rename your cluster column or update the code."
        )

    # Simple QCT flag if exists
    if "is_qct" in gdf.columns:
        gdf["is_qct"] = gdf["is_qct"].astype(bool)
    else:
        gdf["is_qct"] = False

    return gdf


def create_dashboard(gdf: gpd.GeoDataFrame) -> pn.layout.Panel:
    """
    Build the full interactive dashboard.
    """

    # ---------------------------
    # Widgets
    # ---------------------------
    indicator_options = [
        ("E2SFCA (all providers)", "e2sfca_all"),
        ("E2SFCA (Medicaid only)", "e2sfca_medicaid"),
        ("Equity gap (Medicaid penalty)", "equity_gap"),
        ("Priority index (gap × SVI)", "priority_index"),
        ("Social Vulnerability Index (SVI)", "svi"),
        ("Drive time to nearest provider", "travel_time_nn"),
        ("ML prediction (outcome score)", "ml_prediction"),
    ]

    indicator_select = pn.widgets.Select(
        name="Indicator",
        options=dict(indicator_options),
        value="e2sfca_all",
    )

    cluster_values = sorted(gdf["cluster_label"].unique())
    cluster_select = pn.widgets.MultiChoice(
        name="Clusters to show",
        options=cluster_values,
        value=cluster_values,
    )

    qct_toggle = pn.widgets.Checkbox(
        name="Only show Qualified Census Tracts (QCT)",
        value=False,
    )

    # Dynamic range slider based on current indicator
    def _initial_range(col: str):
        col_series = gdf[col].dropna()
        return (
            float(col_series.quantile(0.05)),
            float(col_series.quantile(0.95)),
        )

    init_min, init_max = _initial_range("e2sfca_all")

    value_range_slider = pn.widgets.RangeSlider(
        name="Indicator value range",
        start=float(gdf["e2sfca_all"].min()),
        end=float(gdf["e2sfca_all"].max()),
        value=(init_min, init_max),
    )

    priority_threshold = pn.widgets.FloatSlider(
        name="Priority index threshold",
        start=float(gdf["priority_index"].min()),
        end=float(gdf["priority_index"].max()),
        value=float(gdf["priority_index"].quantile(0.8)),
        step=0.01,
    )

    # ---------------------------
    # Helper filter function
    # ---------------------------
    def filter_gdf(
        indicator: str,
        value_range,
        clusters,
        only_qct: bool,
    ) -> gpd.GeoDataFrame:
        df = gdf.copy()
        vmin, vmax = value_range

        if indicator in df.columns:
            df = df[df[indicator].between(vmin, vmax)]

        if clusters:
            df = df[df["cluster_label"].isin(list(clusters))]

        if only_qct:
            df = df[df["is_qct"]]

        return df

    # ---------------------------
    # Map: spatial distribution of selected indicator
    # ---------------------------
    @pn.depends(
        indicator_select,
        value_range_slider,
        cluster_select,
        qct_toggle,
    )
    def indicator_map(indicator, value_range, clusters, only_qct):
        col = indicator
        df = filter_gdf(col, value_range, clusters, only_qct)

        if df.empty:
            return pn.pane.Markdown(
                "No areas match the current filters.",
                style={"color": "red"},
            )

        return df.hvplot(
            geo=True,
            tiles="CartoLight",
            line_color="white",
            line_width=0.1,
            alpha=0.9,
            column=col,
            cmap="Viridis",
            colorbar=True,
            frame_width=700,
            frame_height=550,
            title=f"Spatial distribution of {col}",
            hover_cols=[
                "tract_id",
                "name",
                "cluster_label",
                "e2sfca_all",
                "e2sfca_medicaid",
                "equity_gap",
                "priority_index",
                "svi",
                "travel_time_nn",
            ],
        )

    # ---------------------------
    # Map: cluster typologies
    # ---------------------------
    def cluster_map():
        return gdf.hvplot(
            geo=True,
            tiles="CartoLight",
            c="cluster_label",
            cmap="Category20",
            line_color="white",
            line_width=0.1,
            alpha=0.9,
            frame_width=700,
            frame_height=550,
            title="Cluster typologies: vulnerability and accessibility",
            hover_cols=[
                "tract_id",
                "name",
                "cluster_label",
                "priority_index",
                "equity_gap",
                "svi",
            ],
        )

    # ---------------------------
    # Map: ML prediction surface
    # ---------------------------
    def ml_map():
        if "ml_prediction" not in gdf.columns:
            return pn.pane.Markdown(
                "Machine learning predictions are not available in this dataset.",
                style={"color": "red"},
            )

        return gdf.hvplot(
            geo=True,
            tiles="CartoLight",
            column="ml_prediction",
            cmap="Inferno",
            colorbar=True,
            line_color="white",
            line_width=0.1,
            alpha=0.9,
            frame_width=700,
            frame_height=550,
            title="Machine learning prediction surface",
            hover_cols=[
                "tract_id",
                "name",
                "cluster_label",
                "priority_index",
                "svi",
            ],
        )

    # ---------------------------
    # Scatter: SVI vs. indicator (equity view)
    # ---------------------------
    @pn.depends(
        indicator_select,
        value_range_slider,
        cluster_select,
        qct_toggle,
    )
    def equity_scatter(indicator, value_range, clusters, only_qct):
        col = indicator
        df = filter_gdf(col, value_range, clusters, only_qct)

        if df.empty:
            return pn.pane.Markdown(
                "No areas match the current filters.",
                style={"color": "red"},
            )

        return df.hvplot.scatter(
            x="svi",
            y=col,
            by="cluster_label",
            legend="top_right",
            frame_width=700,
            frame_height=450,
            title=f"{col} vs. Social Vulnerability Index",
            hover_cols=[
                "tract_id",
                "name",
                "priority_index",
                "equity_gap",
                "travel_time_nn",
            ],
        )

    # ---------------------------
    # Table: top priority tracts for siting
    # ---------------------------
    @pn.depends(priority_threshold)
    def priority_table(threshold):
        df = gdf.copy()
        df = df[df["priority_index"] >= threshold]
        df = df.sort_values("priority_index", ascending=False)

        cols = [
            "tract_id",
            "name",
            "cluster_label",
            "priority_index",
            "equity_gap",
            "svi",
            "e2sfca_medicaid",
            "travel_time_nn",
            "is_qct",
        ]
        cols = [c for c in cols if c in df.columns]

        if df.empty:
            return pn.pane.Markdown(
                "No tracts exceed the current priority threshold.",
                style={"color": "orange"},
            )

        return pn.widgets.Tabulator(
            df[cols],
            layout="fit_data_fill",
            height=350,
            pagination="local",
            page_size=15,
        )

    # ---------------------------
    # Controls panel
    # ---------------------------
    controls = pn.WidgetBox(
        "<b>Controls</b>",
        indicator_select,
        value_range_slider,
        cluster_select,
        qct_toggle,
        "<b>Priority for siting</b>",
        priority_threshold,
        sizing_mode="stretch_width",
        width=330,
    )

    # ---------------------------
    # Tabs with views
    # ---------------------------
    tabs = pn.Tabs(
        (
            "Accessibility map",
            indicator_map,
        ),
        (
            "Equity scatter (SVI vs indicator)",
            equity_scatter,
        ),
        (
            "Cluster map",
            cluster_map,
        ),
        (
            "ML prediction map",
            ml_map,
        ),
        (
            "Priority table (Gap × SVI)",
            priority_table,
        ),
    )

    layout = pn.Row(controls, tabs, sizing_mode="stretch_both")
    return layout


def save_static_html(dash):
    """
    Export the dashboard to docs/dashboard.html for GitHub Pages.
    """
    output_dir = Path("docs")
    output_dir.mkdir(parents=True, exist_ok=True)
    output_file = output_dir / "dashboard.html"

    pn.io.save(
        dash,
        filename=str(output_file),
        title="Healthcare Accessibility and Equity Dashboard",
        resources="cdn",
        embed=False,
    )


if __name__ == "__main__":
    gdf_data = load_data()
    dashboard = create_dashboard(gdf_data)
    # Make it servable for panel serve
    dashboard.servable("Healthcare Accessibility and Equity Dashboard")
    # Save static HTML for GitHub Pages
    save_static_html(dashboard)


DataSourceError: data\processed\final_accessibility.geojson: No such file or directory